In [1]:
from pathlib import Path
parent_dir = str(Path.cwd().parent)
%cd $parent_dir

D:\OrenRichter\Research\pyERGM


D:\OrenRichter\Research\pyERGM\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from utils import *
from ergm import ERGM, BruteForceERGM
from metrics import *

import pandas as pd
import numpy as np

In [3]:
n = 4
p = 0.25
is_directed = False

np.random.seed(98736451)
num_pos_connect = n * (n - 1)

if not is_directed:
    num_pos_connect //= 2

ground_truth_num_edges = round(num_pos_connect * p)
ground_truth_p = ground_truth_num_edges / num_pos_connect
ground_truth_theta = np.array([np.log(ground_truth_p / (1 - ground_truth_p))])

print(f"Ground truth number of edges - {ground_truth_num_edges}")
print(f"Ground truth p - {ground_truth_p}")
print(f"Ground truth theta - {ground_truth_theta}")

adj_mat_no_diag = np.zeros(num_pos_connect)
on_indices = np.random.choice(num_pos_connect, size=ground_truth_num_edges, replace=False).astype(int)
adj_mat_no_diag[on_indices] = 1
adj_mat = np.zeros((n, n))

if not is_directed:
    upper_triangle_indices = np.triu_indices(n, k=1)
    adj_mat[upper_triangle_indices] = adj_mat_no_diag
    lower_triangle_indices_aligned = (upper_triangle_indices[1], upper_triangle_indices[0])
    adj_mat[lower_triangle_indices_aligned] = adj_mat_no_diag
else:
    adj_mat[~np.eye(n, dtype=bool)] = adj_mat_no_diag

print(f"W = ")
print(adj_mat)

number_of_edges_metric = NumberOfEdgesDirected() if is_directed else NumberOfEdgesUndirected()
model = BruteForceERGM(n, [number_of_edges_metric], is_directed=is_directed)
model.fit(adj_mat)
print(f"fit theta: {model._thetas}")
print(model._normalization_factor)

Ground truth number of edges - 2
Ground truth p - 0.3333333333333333
Ground truth theta - [-0.69314718]
W = 
[[0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
optimization started
iteration: 1, time from start training: 0.031241655349731445 log likelihood: -4.159793098485808
fit theta: [0.0348341]
147.9394974344226


In [4]:
n = 4
number_of_edges_metric = NumberOfEdgesDirected() if is_directed else NumberOfEdgesUndirected()
fitted_model = ERGM(n, [number_of_edges_metric], is_directed=is_directed, n_networks_for_grad_estimation=10, n_mcmc_steps=200)
fitted_model.fit(adj_mat)

optimization started
Step 0 - true_grad: 0, grad: [0.], window_grad: 2.10 lr: 0.0010000000, thetas: [0.77766234], time from start: 0.00, n_networks_for_grad_estimation: 10, sliding_grad_window_k: 10
Step 100 - true_grad: 0, grad: [2.], window_grad: 1.70 lr: 0.0009900000, thetas: [0.6242352], time from start: 0.26, n_networks_for_grad_estimation: 10, sliding_grad_window_k: 11
Step 200 - true_grad: 0, grad: [1.8], window_grad: 1.59 lr: 0.0009801000, thetas: [0.47835757], time from start: 0.52, n_networks_for_grad_estimation: 10, sliding_grad_window_k: 12
Step 300 - true_grad: 0, grad: [1.3], window_grad: 1.59 lr: 0.0009702990, thetas: [0.35211616], time from start: 0.80, n_networks_for_grad_estimation: 10, sliding_grad_window_k: 13
Step 400 - true_grad: 0, grad: [0.8], window_grad: 1.17 lr: 0.0009605960, thetas: [0.24716558], time from start: 1.08, n_networks_for_grad_estimation: 10, sliding_grad_window_k: 14
Step 500 - true_grad: 0, grad: [0.8], window_grad: 1.13 lr: 0.0009509900, theta

(array([[ 2.1],
        [ 2.1],
        [ 1.5],
        [ 1.7],
        [ 1.7],
        [ 1.6],
        [ 1.5],
        [ 1.9],
        [ 2. ],
        [ 1.4],
        [ 1.6],
        [ 2. ],
        [ 2.4],
        [ 1.8],
        [ 1.7],
        [ 2.4],
        [ 1.7],
        [ 1.6],
        [ 1.9],
        [ 2.1],
        [ 1.8],
        [ 2.5],
        [ 2.5],
        [ 1.5],
        [ 2.2],
        [ 1.4],
        [ 1.6],
        [ 2.2],
        [ 1.5],
        [ 1.7],
        [ 1.1],
        [ 1.8],
        [ 1.8],
        [ 1.7],
        [ 1.8],
        [ 2.5],
        [ 1.3],
        [ 1.6],
        [ 1.8],
        [ 1.6],
        [ 2.4],
        [ 2. ],
        [ 1.8],
        [ 2.2],
        [ 2.6],
        [ 2.1],
        [ 2. ],
        [ 1.9],
        [ 2.5],
        [ 1.1],
        [ 1.9],
        [ 1.5],
        [ 2.6],
        [ 1.3],
        [ 2.6],
        [ 1.7],
        [ 2. ],
        [ 1.8],
        [ 2. ],
        [ 1.3],
        [ 1.9],
        [ 1.9],
        

In [ ]:
number_of_edges_metric = NumberOfEdgesDirected() if is_directed else NumberOfEdgesUndirected()
model = BruteForceERGM(n, [number_of_edges_metric], is_directed=is_directed)
model.fit(adj_mat)

# print(f"ground truth theta: {ground_truth_theta}")
# print(f"fit theta: {model._thetas}")

# for t_model, t_ground_truth in zip(model._thetas, ground_truth_theta):
#     self.assertAlmostEqual(t_model, t_ground_truth, places=5)

# non_synapses_indices = np.where(adj_mat_no_diag == 0)[0]
# prediction = ground_truth_p * np.ones(adj_mat_no_diag.size)
# prediction[non_synapses_indices] = 1 - ground_truth_p
# true_log_like = np.log(prediction).sum()
# print(f"true log likelihood: {true_log_like}")

# model_with_true_theta = BruteForceERGM(n, [NumberOfEdges()], is_directed=is_directed),
#                                         initial_thetas=np.array(ground_truth_theta), is_directed=is_directed)

# ground_truth_model_log_like = np.log(model_with_true_theta.calculate_weight(adj_mat)) - np.log(
#     model_with_true_theta._normalization_factor)

# print(f"model with true theta log like: {ground_truth_model_log_like}")